### 위성 이미지 가져오기
- 영상구분
- 적외영상(ir105)
- 가시영상(vi006)
- 수증기영상(wv069)
- 단파적외영상(sw038)
- RGB 컬러(rgbt)
- RGB 주야간합성(rgbdn)

In [ ]:
import requests
import json
import datetime
import pandas as pd

In [ ]:
def plus_9(year, month, day, hour, minute):
    date = datetime.datetime(year=year, month=month, day=day, hour=hour, minute=minute)
    date = date + datetime.timedelta(hours=9)
    return (int(date.year), int(date.month), int(date.day), int(date.hour), int(date.minute))

def getImagesDf(img_type): # rgbt
    service_Key = '0f34cad63d39f3e6c937758db43a0fc58300d18c95ade9ad8845163e5e5da5c7'
    url = 'http://apis.data.go.kr/1360000/SatlitImgInfoService/getInsightSatlit'

    date = datetime.datetime.now().date()
    date = datetime.datetime.strftime(date,"%Y%m%d")
    params ={'serviceKey' : service_Key, 'pageNo' : '1', 'numOfRows' : '10', 'dataType' : 'JSON', 'sat' : 'G2', 'data' : img_type, 'area' : 'ko', 'time' : (datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%Y%m%d")  }

    response = requests.get(url, params=params)
    try:
        response.raise_for_status()
    except Exception as e:
        print(e)

    #with open("satellite.csv", "w", encoding="utf8") as f:
    #    f.write(response.text)
    response = json.loads(response.text) # json.loads -> 문자열을 파이썬 객체로 반환.
    images = response["response"]["body"]["items"]["item"][0]["satImgC-file"]
    images_lst = images.replace("[", "").replace("]", "").split(',')


    year = []
    month = []
    day = []
    hour = []
    minute = []

    for image in images_lst:
        y,m,d,H,M = plus_9(int(image[-20:-16]), int(image[-16:-14]), int(image[-14:-12]), int(image[-12:-10]), int(image[-10:-8]))
        year.append(y)
        month.append(m)
        day.append(d) 
        hour.append(H)
        minute.append(M)

    df = pd.DataFrame(images_lst, index=[year,month, day, hour, minute], columns=["이미지 링크"])
    df.index.names = ["Year", "Month", "Day", "Hour", "Minute"]
    return df

def getImageIdx(df, hour, minute):
    date = datetime.datetime.now().date()
    date = datetime.datetime.strftime(date,"%Y-%m-%d")
    date = list(map(int, date.split('-')))

    target = hour * 60 + minute
    idx = 0
    target_idx = -1
    min_score = -1
    for y,m,d,H,M in df.index:
        if date[0] == y and date[1] == m and date[2] == d:
            score = H * 60 + M
            if score > min_score and score <= target:
                min_score = score
                target_idx = idx
        idx += 1
        
    return target_idx
        

df = getImagesDf('rgbt')
idx = getImageIdx(df, 8,30)

if idx == -1:
    print("없다")
else:
    print(df.iloc[idx,0])

 http://www.kma.go.kr/w/repositary/image/sat/gk2a/KO/gk2a_ami_le1b_rgb-true_ko010lc_202511252330.thn.png


### 국토교통부_대중교통 이용인원수

In [36]:
import requests
import json
service_key = 'e08e0ca0df2c2e33cba44a7784c5ec3b096df91b3f038301dcccfc2dcfd53394'
url = "https://apis.data.go.kr/1613000/PublicTransportationPassengerCount/getDailyPublicTransportationPassengerCount"
params = {
    "serviceKey": service_key,
    "pageNo": "1",
    "numOfRows": "10",
    "opr_ymd": "20250101",
    "ctpv_cd": "27",
    "dataType": "JSON"
}

response = requests.get(url, params=params)
with open("sungku.csv", "w", encoding="utf8") as f:
    f.write(response.text)

data = json.loads(response.text)

opr_ymd = data["Response"]["body"]["items"]["item"][0]
print(opr_ymd)

{'opr_ymd': '20250101', 'dow_cd': '4', 'dow_nm': '수요일', 'ctpv_cd': '27', 'ctpv_nm': '대구광역시', 'sgg_cd': '27000', 'sgg_nm': None, 'emd_cd': '2700000000', 'emd_nm': None, 'users_type_cd': '01', 'users_type_nm': '일반인', 'rte_id': '000046', 'sttn_id': '~', 'trfc_mns_se_cd': 'B', 'utztn_nope': 20}


In [1]:
import urllib.request

In [2]:
ServiceKey="843107c0957f6bcc968e6f505aeb6d6f41e5116cb8d1b80ec28d954f3a4d6728"

In [ ]:
#[CODE 1]
def getRequestUrl(url):    
    req = urllib.request.Request(url)    
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None



In [ ]:
#[CODE 2]
def getTourismStatsItem(yyyymm, national_code, ed_cd):    
    service_url = "http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList"
    parameters = "?_type=json&serviceKey=" + ServiceKey   #인증키
    parameters += "&YM=" + yyyymm
    parameters += "&NAT_CD=" + national_code
    parameters += "&ED_CD=" + ed_cd
    url = service_url + parameters
    print(url) #액세스 거부 여부 확인용 출력    
    responseDecode = getRequestUrl(url)   #[CODE 1]
    
    if (responseDecode == None):
        return None
    else:
         return json.loads(responseDecode)
